### Практическая работа № 2
#### Жарова Мария, группа Б05-903

### Предварительные сведения

Векторные нормы:

$||u||_{\infty} = \max_i|u_i|$

$||u||_1 = \sum_i |u_i|$

$||u||_2 = \left(\sum_i |u_i|^2 \right)^{\frac{1}{2}}$

Матричные нормы:

$||A||_{\infty} = \max_i \sum_j |a_{ij}|$

$||A||_1 = \max_j \sum_i |a_{ij}|$

$||A||_2 = \left(\max_i \lambda_i(A A^*) \right)^{\frac{1}{2}}$

Контрольный вопрос: какова будет вторая норма матрицы, если матрица самосопряженная?

Ваш ответ на контрольный вопрос: Если $A_{n\times n}$ - самосопряжённая, то $||A||_2 = max|\lambda _i|,$ где $1\leq i\leq n.$

In [73]:
import numpy as np
import numpy.linalg as la

A = np.array([[1,2],[3,4]], dtype='float')
v = range(0,3)
Vander = np.vander(v)
print('norm_1 = ', la.norm(Vander, 1))
print('norm_2 = ', la.norm(Vander, 2))
print('norm_inf = ', la.norm(Vander, np.inf))
Vander

norm_1 =  5.0
norm_2 =  4.844958524498339
norm_inf =  7.0


array([[0, 0, 1],
       [1, 1, 1],
       [4, 2, 1]])

Обусловленность:
$$(A+\delta A)u = f + \delta f$$
$$\frac{||\delta u||}{||u||}\le \frac{\mu}{1-\mu\frac{||\delta A||}{||A||}} \left(\frac{||\delta f||}{||f||}+\frac{||\delta A||}{||A||}\right)$$

$\mu$ - число обусловленности матрицы A, $\mu(A) = ||A^{-1}||\cdot||A||$, $\mu \ge 1$.



## Пример проблемы использования метода Гаусса для решения СЛАУ

In [1]:
import numpy as np
import numpy.linalg as la

def gauss( A, b ):
    n = b.size
    for k in range(0,n-1):
        for i in range(k+1,n):
            if A[i,k]!=0:
                c = A[i,k]/A[k,k]
                A[i,k+1:n] = A[i,k+1:n] - c*A[k,k+1:n]
                b[i] = b[i] - c*b[k]
                
    # обратный ход
    for k in range(n-1,-1,-1):
        b[k] = (b[k] - np.dot(A[k,k+1:n],b[k+1:n]))/A[k,k];
    return b

#все числа в представлены как вещественные
A1 = np.array([[1e-16, 1., -1.], [-1., 2., -1.], [2., -1., 0.]]);
b1 = np.array([0., 0., 1.]);

A2 = np.array([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
b2 = np.array([1., 0., 0.])

In [2]:
print(A1)

[[ 1.e-16  1.e+00 -1.e+00]
 [-1.e+00  2.e+00 -1.e+00]
 [ 2.e+00 -1.e+00  0.e+00]]


In [6]:
print(A2)

[[ 2.e+00 -1.e+00  0.e+00]
 [-1.e+00  2.e+00 -1.e+00]
 [ 1.e-16  1.e+00 -1.e+00]]


In [4]:
b2 = np.array([1., 0., 0.])
b2.shape[0]

3

In [3]:
A1 = np.array([[1e-16, 1., -1.], [-1., 2., -1.], [2., -1., 0.]]);
b1 = np.array([0., 0., 1.]);

A2 = np.array([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
b2 = np.array([1., 0., 0.])

print('mu1 = ', la.cond(A1))
print('mu2 = ', la.cond(A2))

print('u1 = ', gauss(A1, b1))
#print('u1 = ', la.solve(A1, b1))
print('u2 = ', gauss(A2, b2))#la.solve(A2, b2))

mu1 =  16.39373162228438
mu2 =  16.393731622284395
u1 =  [0.55511151 0.25       0.25      ]
u1 =  [2.77555756e+15 2.77555756e-01 1.80111512e-17]
u2 =  [1. 1. 1.]


## Часть 1. LU разложение

Задание: 

реализовать алгоритм решения предыдущей задачи с матрицей A2 с помощью LU-разложение В решении должна выводиться L, U и собственно решение системы. 

ВАЖНО: реализация метода LU должна быть получена путем небольшой модификации метода gauss!  При это саму реализацию можно разделить на два метода: один метод собственно находит LU разложение (можно сделать переделкой цикла для матрицы A метода gauss), второй метод - непосредственное решение системы с помощью прямого и обратного хода. Ни в каком виде нельзя ползоваться пакетными методами (в частности, la.solve)

In [194]:
import numpy as np

In [195]:
""" 
    Input: A - матрица, для которой будем делать LU-разложение.
    Output: матрицы LU, L и U соответственно.
    Функция находит LU-разложение для входной матрицы, если оно существует.
"""
def get_LU(A):
    # проверим, что LU-разложение существует:
    # используем критерий, что в A все верхние подматрицы должны быть невырожденны.
    min_size = min(A.shape[0], A.shape[1])
    for i in range(min_size):
        if np.linalg.det(A[:i+1, :i+1]) == 0:
            raise Exception('Для данной матрицы LU-разложения не существует.')
    
    # В случае успеха модифицируем алгоритм Гаусса:
    # L - верхнетреугольная матрица с 1 на ГД, U - нижнетреугольная,
    # составим LU как сумму U и L с занулённой ГД.
    LU = np.matrix(np.zeros([A.shape[0], A.shape[1]]))
    n = A.shape[0]
    for k in range(n):
        for i in range(k, n):
            LU[k, i] = A[k, i] - LU[k, :k] * LU[:k, i]
        for i in range(k + 1, n):
            LU[i, k] = (A[i, k] - LU[i, :k] * LU[:k, k]) / LU[k, k]
    # теперь легко выделить L и U:
    L = LU.copy()
    U = LU.copy()
    for i in range(n):
        L[i, i] = 1
        L[i, i+1:] = 0
    for i in range(1, n):
        U[i, :i] = 0
            
    return LU, L, U

In [196]:
A = np.matrix([[2,3],[5,4]])#, dtype='float')
ans = get_LU(A)
display(ans)
ans[1] @ ans[2] == A

(matrix([[ 2. ,  3. ],
         [ 2.5, -3.5]]),
 matrix([[1. , 0. ],
         [2.5, 1. ]]),
 matrix([[ 2. ,  3. ],
         [ 0. , -3.5]]))

matrix([[ True,  True],
        [ True,  True]])

In [162]:
""" 
    Input: LU - LU-матрица, для матрицы A системы Ax = b (полученная из функции get_LU);
           b - вектор-столбец свободных членов системы.
    Output: решение x системы.
    Функция решает СЛАУ, используя LU-разложение:
    - Ax = b;
    - LUx = b;
    - Ly = b - решается прямой подстановкой;
    - Ux = b - решается обратной подстановкой.
"""
def LU_solve(LU, b):
    n = LU.shape[0]
    # найдём вектор y (прямая подстановка)
    y = np.matrix(np.zeros([n, 1]))
    for i in range(y.shape[0]):
        y[i] = b[i] - LU[i, :i] * y[:i]

    # найдём вектор x (обратная подстановка)
    x = np.matrix(np.zeros([n, 1]))
    for i in range(1, n + 1):
        x[-i] = (y[-i] - LU[-i, -i:] * x[-i:]) / LU[-i, -i]

    return x

In [188]:
b = np.matrix([2, 2]).T
x = LU_solve(ans[0], b)
print('x =', x)
print('--------')
print(A @ x)
print(b)

x = [[-0.28571429]
 [ 0.85714286]]
--------
[[2.]
 [2.]]
[[2]
 [2]]


### LU - разложение с помощью пакета sympy

Чтобы убедиться, что разложение получено верно, можно воспользоваться скриптом ниже

In [100]:
import sympy as sp

A = sp.Matrix([[2, 3], [5, 4]])
L, U, _ = A.LUdecomposition()
display(L)
display(U)
L*U

Matrix([
[  1, 0],
[5/2, 1]])

Matrix([
[2,    3],
[0, -7/2]])

Matrix([
[2, 3],
[5, 4]])

## Часть 2. Нахождение обратной матрицы с помощью LU разложения

Задание:

Предложить алгоритм с использованием LU-разложения и найти обратную матрицу с точностью $\epsilon = 10^{-3}$:
$$
A = \begin{pmatrix} 
1 & 1  & 1 \\
0 & 1 & 2 \\
7 & 1 & 4 \\
\end{pmatrix}
$$
Для необходимых оценок использовать первую норму. Сравнить результат со значением, найденным с помощью функции numpy.inv.

In [230]:
import numpy as np
from sklearn.metrics import precision_score, accuracy_score

In [231]:
""" 
    Input: LU - LU-матрица для матрицы A (полученная из функции get_LU).
    Output: A^{-1} - обратная матрица к А.
    Функция находит обратную матрицу, используя LU-разложение исходной
    (будем последовательно находить каждый столбец x обратной матрицы как решение 
    системы LUx = e, где e - соответствующий столбец единичной матрицы).
"""
def LU_inverse(LU):
    n = LU.shape[0]
    E = np.identity(n)
    A_inv = np.zeros((n,n))
    e = np.zeros(n)
    # составим итоговую A^{-1} из отдельно полученных векторов
    for i in range(n):
        # составляем столбец e
        for j in range(n):
            e[j] = E[i][j]    
        col = LU_solve(LU, e) # находим столбец обратной матрицы
        # вставляем его в итоговый ответ
        for j in range(n):
            A_inv[j][i] = col[j]

    return A_inv

A = np.matrix([[1, 1, 1],[0, 1, 2], [7, 1, 4]])
A_LU = get_LU(A)
my_inv = LU_inverse(A_LU[0])
my_inv

array([[ 0.22222222, -0.33333333,  0.11111111],
       [ 1.55555556, -0.33333333, -0.22222222],
       [-0.77777778,  0.66666667,  0.11111111]])

In [232]:
true_inv = np.linalg.inv(A)
true_inv

matrix([[ 0.22222222, -0.33333333,  0.11111111],
        [ 1.55555556, -0.33333333, -0.22222222],
        [-0.77777778,  0.66666667,  0.11111111]])

In [238]:
print(np.linalg.norm(my_inv, ord=1))
print(np.linalg.norm(true_inv, ord=1))

2.5555555555555554
2.5555555555555554
